In [1]:
import time
import os
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["TF_ENABLE_ONEDNN_OPTS"]="1"
# tf.config.set_visible_devices([], 'GPU')
T=3.0
N=int(T*2*12)
r=0.0303
d=0
Nsim = 10
dtype=np.float32
seed=1

t=np.linspace(0,T,N*Nsim,endpoint=True,dtype=dtype).reshape((-1,1))
t=tf.constant(t)

from Commodities import Schwartz2Factor
'Salmon'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.01, 0.9, 0.57, 95] # down,down
salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.2, 0.9, 0.57, 95] # down,up
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.6, 0.9, 0.57, 95] # up,up

'Soy'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# soyParam=[0.15, 0.5, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1500] # low vol
soyParam=[0.15, 1, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # medium vol
# soyParam=[0.15, 2, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1500] # high vol

'Risk neutral dynamics'
salmonParam[0]=r
soyParam[0]=r

"Fish feeding 25% of production cost, disease 30%, harvest 10%. Total production cost = 50% of price = labor, smolt, ..."
salmonPrice=salmonParam[-1] #NOK/KG
harvestingCosts=salmonPrice*0.5*0.1 # roughly 10%
feedingCosts=salmonPrice*0.5*0.25
initialSalmon=0.5*salmonPrice+feedingCosts+harvestingCosts #we add the costs to salmon price since they are respected in the model, other costs are fixed and thus removed
salmonParam[-1]=initialSalmon
print(f'Feeding costs {feedingCosts} and Harvesting costs {harvestingCosts}')
# soyParam[-1]=feedingCosts # to save the right dataset, since initial price is not relevant for soy model


soy=Schwartz2Factor(soyParam,t,dtype=dtype)
salmon=Schwartz2Factor(salmonParam,t,dtype=dtype)


from Harvest import Harvest
hc = harvestingCosts
harvest = Harvest(hc)

from Growth import Bertalanffy
wInf=6
a=1.113
b=1.097
c=1.43
growth = Bertalanffy(t,wInf,a,b,c)

from Price import Price
price = Price(salmon)

from Feed import StochFeed,DetermFeed
cr=1.1
fc=feedingCosts
feed = StochFeed(fc,cr,r,t,soy)
# feed = DetermFeed(fc,cr,r,t,soy)

from Mortality import ConstMortatlity,HostParasite
# n0=10000
# m=0.1
# mort = ConstMortatlity(t,n0,m)

params=[0.05,0.1,8.71,0.05]
beta=[0.0835,0.0244]
H0=10000.0
P0=1
mort = HostParasite(t,params,beta,H0,P0)

from FishFarm import fishFarm
farm = fishFarm(growth,feed,price,harvest,mort,stride=Nsim,seed=seed)


Feeding costs 11.875 and Harvesting costs 4.75


c:\ProgramData\Anaconda3\envs\intelpython310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
farm.seed(seed)

batch_size=2**12
batches=20
gen = farm.generateFishFarm

from OptimalStopping import Polynomial,DeepOptS,LSMC
basis = Polynomial(deg=2,dtype=dtype)
opt=LSMC(r,farm.tCoarse,gen,basis,batch_size=batch_size)


opt.train(batches)

farm.seed(seed+1)
tic=time.time()
tau,Vtau=opt.evaluate(batches)
ctimeEval=time.time()-tic

print(tau.shape)
print(f'Mean stopping time {np.mean(tau)} with mean value {np.mean(Vtau)} in {ctimeEval} s')

(81920,)
Mean stopping time 1.9331333637237549 with mean value 1948095.625 in 13.71017599105835 s


In [3]:
farm.seed(seed)

batch_size=2**12
batches=20
gen = farm.generateFishFarm

from OptimalStopping import Polynomial,DeepOptS,LSMC

opt=DeepOptS(r,farm.tCoarse,gen,d=farm.d,batch_size=batch_size)

opt.train(batches)

farm.seed(seed+1)
tic=time.time()
tau,Vtau=opt.evaluate(batches)
ctimeEval=time.time()-tic

print(tau.shape)
print(f'Mean stopping time {np.mean(tau)} with mean value {np.mean(Vtau)} in {ctimeEval} s')

  0%|          | 0/3006 [00:00<?, ?epoch/s]

InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: `generator` yielded an element of shape (4096, 8, 146) where an element of shape (4096, 7, 146) was expected.
Traceback (most recent call last):

  File "c:\ProgramData\Anaconda3\envs\intelpython310\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "c:\ProgramData\Anaconda3\envs\intelpython310\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\ProgramData\Anaconda3\envs\intelpython310\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1047, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (4096, 8, 146) where an element of shape (4096, 7, 146) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) INVALID_ARGUMENT:  TypeError: `generator` yielded an element